## We are going to solve a customer classification Problem. The problem can be summarized as below:

1. Given the details about the customer our objective is to identify which customers is a potential high revenue customers and which customers are low revenue customers for a stock brocking firm
2. The problem is a supervised classification problem

Machine Learning Pipleline:

1. Load the Data into python
2. Split the data into train and val (only when validation set is not provided separately)
3. Exploratory Data Analysis and Preprocessing
4. Model Building
5. Model Evaluation
6. Iter over to improve the performance

In [79]:
###### Importing Libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pickle

## Load the Dataset

In [80]:
raw_data = pd.read_csv('Existing_Base.csv')

In [81]:
###### Info gives details about the data #####

raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10155 entries, 0 to 10154
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   REF_NO                           10155 non-null  int64  
 1   children                         10155 non-null  object 
 2   age_band                         10155 non-null  object 
 3   status                           10155 non-null  object 
 4   occupation                       10155 non-null  object 
 5   occupation_partner               10155 non-null  object 
 6   home_status                      10155 non-null  object 
 7   family_income                    10155 non-null  object 
 8   self_employed                    10155 non-null  object 
 9   self_employed_partner            10155 non-null  object 
 10  year_last_moved                  10155 non-null  int64  
 11  TVarea                           10155 non-null  object 
 12  post_code         

In [82]:
##### Check the class imbalance in the data. This is a common problem in classification problem
##### This tells us that our data has class imbalance

raw_data['Revenue Grid'].value_counts(normalize=True)

2    0.893058
1    0.106942
Name: Revenue Grid, dtype: float64

## Split the data into training and validation set
We would use sklearn's `train_test_split` functionality for the same.

In [83]:
targets = raw_data['Revenue Grid']
features = raw_data.drop(['Revenue Grid'], axis=1)  ## This line drops Revenue Grid from the data

features_train, features_val, target_train, target_val = train_test_split(
    features, targets, test_size=0.2, random_state=42, stratify=targets)

In [84]:
features_train.reset_index(drop=True, inplace=True)
features_val.reset_index(drop=True, inplace=True)
target_train.reset_index(drop=True, inplace=True)
target_val.reset_index(drop=True, inplace=True)

In [102]:
features_val.to_csv('features_val.csv', index=None)
target_val.to_csv('target_val.csv', index=None)

In [85]:
features_train.shape, features_val.shape

((8124, 31), (2031, 31))

In [86]:
target_val.value_counts(normalize=True)

2    0.893156
1    0.106844
Name: Revenue Grid, dtype: float64

## Explore the data!!!

1. Missing value imputations
2. Convert all non numeric to numeric columns
3. If there are any additional issues with the data

In [87]:
##### Get the head of the data #####
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
features_train.head(20)

<ipython-input-87-54a650fc0b73>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,year_last_moved,TVarea,post_code,post_area,Average Credit Card Transaction,Balance Transfer,Term Deposit,Life Insurance,Medical Insurance,Average A/C Balance,Personal Loan,Investment in Mutual Fund,Investment Tax Saving Bond,Home Loan,Online Purchase Amount,gender,region,Investment in Commudity,Investment in Equity,Investment in Derivative,Portfolio Balance
0,6756,2,31-35,Partner,Housewife,Other,Own Home,">=35,000",No,No,1988,Granada,WA4 4QG,WA4,64.48,29.49,9.99,47.96,7.49,18.46,5.99,19.98,0.00,8.99,0.00,Female,North West,31.88,8.90,16.65,53.28
1,5618,1,31-35,Partner,Housewife,Professional,Own Home,"<30,000, >=27,500",No,No,1995,Carlton,CM16 6DW,CM16,0.00,174.47,21.48,97.94,52.16,34.93,0.00,89.43,5.98,7.98,0.00,Female,South East,69.21,23.05,46.74,127.76
2,6486,2,31-35,Partner,Housewife,Professional,Own Home,"<27,500, >=25,000",No,No,1987,Ulster,BT23 7BN,BT23,30.48,19.49,0.52,50.94,21.98,5.99,11.99,17.48,4.99,0.00,0.00,Female,Northern Ireland,24.68,6.74,18.90,62.98
3,4557,1,51-55,Partner,Housewife,Professional,Own Home,">=35,000",No,Yes,1982,Central,B45 8LX,B45,0.01,0.00,0.00,0.00,0.00,0.00,111.95,0.00,0.00,0.00,0.00,Female,West Midlands,0.00,18.66,18.66,48.14
4,2450,3,26-30,Partner,Other,Manual Worker,Own Home,"<25,000, >=22,500",No,No,1988,Granada,WA5 1XU,WA5,0.00,0.03,0.00,192.85,0.00,71.97,0.00,48.43,0.00,0.00,0.00,Female,North West,38.58,20.07,52.21,132.89
5,821,2,26-30,Partner,Other,Professional,Own Home,"<27,500, >=25,000",Yes,No,1987,Granada,FY5 2ER,FY5,0.00,79.47,0.00,61.47,0.00,29.99,0.00,11.99,0.00,9.98,0.00,Male,North West,28.19,8.66,17.24,79.22
6,10382,Zero,31-35,Single/Never Married,Secretarial/Admin,Unknown,Own Home,"<27,500, >=25,000",No,No,1997,Scottish TV,EH26 0QU,EH26,0.00,0.00,19.99,9.49,0.00,8.99,0.00,0.00,0.00,0.00,0.00,Female,Scotland,5.90,1.50,3.08,-10.51
7,8316,Zero,51-55,Widowed,Retired,Unknown,Own Home,"< 4,000",No,No,1993,Yorkshire,YO13 9LR,YO13,0.00,29.47,10.00,11.94,4.99,9.43,0.00,10.95,0.00,0.00,0.00,Female,Unknown,11.28,3.40,6.22,-9.31
8,10196,Zero,41-45,Partner,Housewife,Unknown,Own Home,"<10,000, >= 8,000",No,Yes,1993,HTV,SN15 4HD,SN15,74.40,34.47,0.01,64.94,64.45,129.93,0.00,98.91,0.00,21.46,0.00,Female,South West,47.65,41.72,59.71,176.27
9,5374,Zero,31-35,Partner,Housewife,Manual Worker,Own Home,"<15,000, >=12,500",No,No,1984,Granada,LA9 7LR,LA9,0.00,0.00,0.00,0.52,0.00,4.49,0.00,0.00,0.00,0.00,0.00,Female,North,0.10,0.75,0.84,-51.04


In [88]:
#### Lets get info on our columns #####

features_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   REF_NO                           8124 non-null   int64  
 1   children                         8124 non-null   object 
 2   age_band                         8124 non-null   object 
 3   status                           8124 non-null   object 
 4   occupation                       8124 non-null   object 
 5   occupation_partner               8124 non-null   object 
 6   home_status                      8124 non-null   object 
 7   family_income                    8124 non-null   object 
 8   self_employed                    8124 non-null   object 
 9   self_employed_partner            8124 non-null   object 
 10  year_last_moved                  8124 non-null   int64  
 11  TVarea                           8124 non-null   object 
 12  post_code           

In [89]:
####### Understand the Object columns ########

##### Understanding the children column #####

### How many unique values
### What is the distribution of each unique values

train_uniq_values = {}

def children_numeric_converter(children_data, unique_values):
  children_data[~(children_data.isin(unique_values))] = "Zero" # This replaces every other value to Zero
  children_data[children_data=='Zero'] = 0  # Converts Zero to numeric 0
  children_data[children_data=='4+'] = 4    # Converts 4+ to 4
  children_data = pd.to_numeric(children_data) # Converts children_data to numeric
  return children_data


features_train['children'].nunique()

features_train['children'].value_counts()

uniq_values = features_train['children'].unique()

children_numeric = children_numeric_converter(features_train['children'], uniq_values)

features_train.drop(['children'], axis=1, inplace=True) ### drops the children column

features_train['children'] = children_numeric

train_uniq_values['children'] = uniq_values.tolist()

<ipython-input-89-b595c7c70012>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  children_data[~(children_data.isin(unique_values))] = "Zero" # This replaces every other value to Zero
<ipython-input-89-b595c7c70012>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  children_data[children_data=='Zero'] = 0  # Converts Zero to numeric 0
<ipython-input-89-b595c7c70012>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  children_data[child

In [90]:
###### Analyzing the Post code column ######
columns_to_remove = []

features_train['post_code'].nunique()
features_train['post_area'].nunique()

## Gives me an impression that I can drop this column

features_train.drop(['post_code', 'post_area', 'REF_NO'], axis=1, inplace=True)
columns_to_remove.extend(['post_code', 'post_area', 'REF_NO'])


In [91]:
###### Analyzing the Age band columns ######

def ageband_numeric_converter(ageband_data, uniq_values):

  def __split(x):
    splits = x.split('-')
    left_val = splits[0]
    right_val = splits[1]
    mean = (float(left_val) + float(right_val))/2
    return mean

  ageband_data[~(ageband_data.isin(uniq_values))] = '45-50'
  ageband_data[ageband_data=='71+'] = '71-75'
  ageband_data[ageband_data=='Unknown'] = '45-50'
  mean_vals = ageband_data.apply(__split)
  return mean_vals


features_train['age_band'].nunique()

features_train['age_band']. value_counts()

ageband_unique = features_train['age_band'].unique()

numeric_age_band = ageband_numeric_converter(features_train['age_band'], ageband_unique)

features_train.drop(['age_band'], axis=1)

features_train['age_band'] = numeric_age_band

train_uniq_values['age_band'] = ageband_unique.tolist()

<ipython-input-91-4bfd66504c88>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ageband_data[~(ageband_data.isin(uniq_values))] = '45-50'
<ipython-input-91-4bfd66504c88>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ageband_data[ageband_data=='71+'] = '71-75'
<ipython-input-91-4bfd66504c88>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ageband_data[ageband_data=='Unknown'] = '45-50'


In [92]:
features_train['gender'].value_counts()

Female     6122
Male       1975
Unknown    27  
Name: gender, dtype: int64

In [93]:
###### Analyzing the Gender column #######

def gender_numeric_converter(gender_data, uniq_values):
  gender_data[~(gender_data.isin(uniq_values))] = 'Female'
  gender_data[gender_data=='Unknown'] = 'Female'
  gender_data[gender_data=='Male'] = 0
  gender_data[gender_data=='Female'] = 1
  gender_data = pd.to_numeric(gender_data)
  return gender_data

gender_uniq = features_train['gender'].unique()

gender_numeric = gender_numeric_converter(features_train['gender'], gender_uniq)

features_train.drop(['gender'], axis=1, inplace=True)

features_train['gender'] = gender_numeric

train_uniq_values['gender'] = gender_uniq.tolist()

<ipython-input-93-d35e1d30ea44>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_data[~(gender_data.isin(uniq_values))] = 'Female'
<ipython-input-93-d35e1d30ea44>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_data[gender_data=='Unknown'] = 'Female'
<ipython-input-93-d35e1d30ea44>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gender_data[gender_data=='Male'] = 0
<ipython-input-93-d35e1d30ea44>:7: SettingWithCopyWarni

In [94]:
missing_value_replacement = {}
missing_value_replacement['status'] = ['Partner']
missing_value_replacement['occupation'] = ['Professional']
missing_value_replacement['occupation_partner'] = ['Unknown', 'Professional']
missing_value_replacement['home_status'] = ['Own Home']
missing_value_replacement['self_employed'] = ["No"]
missing_value_replacement['self_employed_partner'] = ["No"]
missing_value_replacement['TVarea'] = ["Central", "Carlton", "Meridian"]
missing_value_replacement['region'] = ["South East"]
missing_value_replacement['family_income'] = [">=35,000"]
missing_value_replacement['Average Credit Card Transaction'] = [0.0]
missing_value_replacement['Balance Transfer'] = [16.975]
missing_value_replacement['Life Insurance'] = [30.0]
missing_value_replacement['Medical Insurance'] = [0.0]
missing_value_replacement['Average A/C Balance'] = [15.00]
missing_value_replacement['Personal Loan'] = [0.0]
missing_value_replacement['Investment in Mutual Fund'] = [23.47]
missing_value_replacement['Investment Tax Saving Bond'] = [0.0]
missing_value_replacement['Home Loan'] = [0.0]
missing_value_replacement['Online Purchase Amount'] = [0.0]
missing_value_replacement['Investment in Commudity'] = [23.79]
missing_value_replacement['Investment in Equity'] = [12.82]
missing_value_replacement['Investment in Derivative'] = [20.89]
missing_value_replacement['Portfolio Balance'] = [65.38]

In [95]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 28 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   age_band                         8124 non-null   float64
 1   status                           8124 non-null   object 
 2   occupation                       8124 non-null   object 
 3   occupation_partner               8124 non-null   object 
 4   home_status                      8124 non-null   object 
 5   family_income                    8124 non-null   object 
 6   self_employed                    8124 non-null   object 
 7   self_employed_partner            8124 non-null   object 
 8   year_last_moved                  8124 non-null   int64  
 9   TVarea                           8124 non-null   object 
 10  Average Credit Card Transaction  8124 non-null   float64
 11  Balance Transfer                 8124 non-null   float64
 12  Term Deposit        

In [96]:
####### Finding replacement values for the other columns #######
features_train.describe()

,age_band,year_last_moved,Average Credit Card Transaction,Balance Transfer,Term Deposit,Life Insurance,Medical Insurance,Average A/C Balance,Personal Loan,Investment in Mutual Fund,Investment Tax Saving Bond,Home Loan,Online Purchase Amount,Investment in Commudity,Investment in Equity,Investment in Derivative,Portfolio Balance,children,gender
count,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000
mean,46.533050,1968.430822,23.536013,45.848500,26.857107,64.896397,18.816137,31.832499,25.705244,41.928358,6.055491,4.401269,19.354365,35.991018,21.546958,31.539849,89.368669,0.638971,0.756893
std,13.511666,179.942993,51.008213,79.870499,53.022363,94.681534,32.094002,45.166539,88.237940,64.665153,12.737090,9.809771,93.045927,42.117391,32.826080,39.877961,108.716211,0.916030,0.428985
min,19.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-77.230000,0.000000,0.000000
25%,38.000000,1978.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.090000,4.500000,8.500000,26.375000,0.000000,1.000000
50%,47.500000,1988.000000,0.000000,16.975000,0.000000,30.000000,0.010000,14.990000,0.000000,23.470000,0.000000,0.000000,0.000000,23.790000,12.820000,20.895000,65.385000,0.000000,1.000000
75%,57.500000,1994.000000,23.980000,64.970000,34.490000,92.932500,26.970000,45.952500,20.980000,59.460000,5.490000,4.490000,7.980000,50.127500,27.962500,42.392500,125.015000,1.000000,1.000000
max,73.000000,1999.000000,662.260000,2951.760000,784.820000,2930.410000,591.040000,626.240000,4905.930000,2561.270000,138.560000,162.350000,4306.420000,1231.090000,1279.100000,1771.160000,4283.560000,4.000000,1.000000


In [97]:
##### Analysing the Status column ######

one_hot_converted_columns = ['status', 'occupation', 'occupation_partner', 'home_status', 'self_employed', 'self_employed_partner', 'TVarea', 'region', 'family_income']

encoder = OneHotEncoder()

encoder.fit(features_train[one_hot_converted_columns])

encoded_data = pd.DataFrame(encoder.transform(features_train[one_hot_converted_columns]).toarray())

encoded_data.columns = encoder.get_feature_names_out()

features_train.drop(one_hot_converted_columns, axis=1, inplace=True)

features_train = pd.concat([features_train, encoded_data], axis=1)


In [98]:
##### Analysing the year last moved column #####
columns_to_remove.append('year_last_moved')
features_train.drop(['year_last_moved'], axis=1, inplace=True)


In [103]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 90 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   age_band                              8124 non-null   float64
 1   Average Credit Card Transaction       8124 non-null   float64
 2   Balance Transfer                      8124 non-null   float64
 3   Term Deposit                          8124 non-null   float64
 4   Life Insurance                        8124 non-null   float64
 5   Medical Insurance                     8124 non-null   float64
 6   Average A/C Balance                   8124 non-null   float64
 7   Personal Loan                         8124 non-null   float64
 8   Investment in Mutual Fund             8124 non-null   float64
 9   Investment Tax Saving Bond            8124 non-null   float64
 10  Home Loan                             8124 non-null   float64
 11  Online Purchase A

In [101]:
with open('column_sequence.pickle', 'wb') as f:
  pickle.dump(list(features_train.columns), f)

In [ ]:
###### We need to keep in mind that missing values could be present in the validation and Testing set

### Store the names of the columns we removed
### Store the unique values in the columns
### Store the onehot encoder object

with open('columns_to_remove.pickle', 'wb') as f:
  pickle.dump(columns_to_remove, f)

with open('column_unique_values.pickle', 'wb') as f:
  pickle.dump(train_uniq_values, f)

with open('column_encoder.pickle', 'wb') as f:
  pickle.dump(encoder, f)

In [ ]:
with open('missing_value_replacement.pickle', 'wb') as f:
  pickle.dump(missing_value_replacement, f)